In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-notifications")  # Disable notifications

browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

browser.get("https://www.nasdaq.com")

accept_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'I Accept')]")))

accept_button.click()

browser.implicitly_wait(3)

element = browser.find_element_by_css_selector(".nsdq-c-band.nsdq-u-padding-top-none.nsdq-u-padding-bottom-none.jupiter22-c-stock-ticker__section")
print(element.get_attribute('innerHTML'))

browser.quit()


AttributeError: 'WebDriver' object has no attribute 'find_element_by_css_selector'